<h3><center><a href='https://cambridgespark.com/content/tutorials/implementing-your-own-recommender-systems-in-Python/index.html'>Implementing your own recommender systems in Python</a></center></h3>

<h3><center>Original article from author: Agnes Johannsdottir</center></h3>

Two most ubiquitous types of recommender systems are **Content-Based** and **Collaborative Filtering (CF)**. 

Collaborative filtering produces recommendations based on the knowledge of users’ attitude to items, that is it uses the “wisdom of the crowd” to recommend items. 

In contrast, content-based recommender systems focus on the attributes of the items and give you recommendations based on the similarity between them.

**Collaborative Filtering (CF)** can be divided into *Memory-Based Collaborative Filtering* and *Model-Based Collaborative filtering*. In this tutorial, you will implement Model-Based CF by using singular value decomposition (SVD) and Memory-Based CF by computing cosine similarity.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd

In [3]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=header)

#### http://files.grouplens.org/datasets/movielens/ml-100k-README.txt
`u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.
              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1.  The data is randomly
              ordered. This is a tab separated list of 
	         user id | item id | rating | timestamp. 
              The time stamps are unix seconds since 1/1/1970 UTC`

In [4]:
print('Raw data size:', df.shape, 
      '\nUnique users:', len(df['user_id'].unique()), 
      '\nUnique movies:', len(df['item_id'].unique()), 
      '\nUnique ratings:', len(df['rating'].unique()))
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

Raw data size: (100000, 4) 
Unique users: 943 
Unique movies: 1682 
Unique ratings: 5


In [5]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


You can use the **scikit-learn** library to split the dataset into testing and training. 

**Cross_validation.train_test_split** shuffles and splits the data into two datasets according to the percentage of test examples (test_size), which in this case is 0.25.

In [6]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.25)

/Users/george/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
print('train_data.shape:', train_data.shape, 
      '\ntest_data.shape:', test_data.shape)

train_data.shape: (75000, 4) 
test_data.shape: (25000, 4)


<h3><center>Memory-Based Collaborative Filtering</center></h3>

Memory-Based Collaborative Filtering approaches can be divided into two main sections: **user-item filtering** and **item-item filtering**. A user-item filtering takes a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked. In contrast, item-item filtering will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items and outputs other items as recommendations.

**Example of user-item matrix:**

<img src="User-Item-Matrix.png">

**User-Item Collaborative Filtering:** “Users who are similar to you also liked …”

<img src="User-Item Collaborative Filtering.png">

**Item-Item Collaborative Filtering:** “Users who liked this item also liked …”

<img src="Item-Item Collaborative Filtering.png">

In both cases, you create a user-item matrix which you build from the entire dataset. Since you have split the data into testing and training you will need to create two 943 ×
 1682 matrices. The training matrix contains 75% of the ratings and the testing matrix contains 25% of the ratings.
 
A distance metric commonly used in recommender systems is cosine similarity, where the ratings are seen as vectors in n-dimensional space and the similarity is calculated based on the angle between these vectors.

**Cosine similiarity between users a and k**
<img src="Cosine similiarity between users a and k.png">

**Cosine similiarity between items m and b**
<img src="Cosine similiarity between items m and b.png">

In [8]:
train_data[:3]

,user_id,item_id,rating,timestamp
22071,457,237,4,882393712
73328,894,327,4,881625708
85863,892,465,4,886609295


In [9]:
for line in train_data[:3].itertuples():
    print(line)
    print(line[1]-1, line[2]-1, line[3])

Pandas(Index=22071, user_id=457, item_id=237, rating=4, timestamp=882393712)
456 236 4
Pandas(Index=73328, user_id=894, item_id=327, rating=4, timestamp=881625708)
893 326 4
Pandas(Index=85863, user_id=892, item_id=465, rating=4, timestamp=886609295)
891 464 4


In [10]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [11]:
print(train_data_matrix.shape)
train_data_matrix

(943, 1682)


array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
print(test_data_matrix.shape)
test_data_matrix

(943, 1682)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

You can use the **pairwise_distances** function from sklearn to calculate the **cosine similarity**. 

Note, the output will range from **0 (no distance, identical vectors)** to **1 (significant distance, different vectors)** since the ratings are all positive.

http://scikit-learn.org/stable/modules/metrics.html#metrics

In [13]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [14]:
print(user_similarity.shape)
user_similarity

(943, 943)


array([[0.        , 0.83686172, 0.95417651, ..., 0.82273192, 0.8925198 ,
        0.72052657],
       [0.83686172, 0.        , 0.8655521 , ..., 0.83564478, 0.82333172,
        0.90311035],
       [0.95417651, 0.8655521 , 0.        , ..., 0.90306316, 0.87857412,
        0.98524673],
       ...,
       [0.82273192, 0.83564478, 0.90306316, ..., 0.        , 0.96339721,
        0.93293921],
       [0.8925198 , 0.82333172, 0.87857412, ..., 0.96339721, 0.        ,
        0.83705554],
       [0.72052657, 0.90311035, 0.98524673, ..., 0.93293921, 0.83705554,
        0.        ]])

In [15]:
print(item_similarity.shape)
item_similarity

(1682, 1682)


array([[0.        , 0.68586326, 0.77905449, ..., 1.        , 0.94501426,
        1.        ],
       [0.68586326, 0.        , 0.81880821, ..., 1.        , 0.91266662,
        1.        ],
       [0.77905449, 0.81880821, 0.        , ..., 1.        , 1.        ,
        0.8901622 ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 1.        ,
        1.        ],
       [0.94501426, 0.91266662, 1.        , ..., 1.        , 0.        ,
        1.        ],
       [1.        , 1.        , 0.8901622 , ..., 1.        , 1.        ,
        0.        ]])

<h3><center>Next step is to make **predictions**</center></h3>

You have already created similarity matrices: **user_similarity** and **item_similarity** and therefore you can make a prediction by applying following formula for user-based CF:

You can look at the similarity between **users k** and **a** as weights that are multiplied by the ratings of a similar **user a** (corrected for the average rating of that user). You will need to normalize it so that the ratings stay between 1 and 5 and, as a final step, sum the average ratings for the user that you are trying to predict.

The idea here is that some users may tend always to give high or low ratings to all movies. The relative difference in the ratings that these users give is more important than the absolute values. To give an example: suppose, **user k** gives 4 stars to his favourite movies and 3 stars to all other good movies. Suppose now that another **user t** rates movies that he/she likes with 5 stars, and the movies he/she fell asleep over with 3 stars. These two users could have a very similar taste but treat the rating system differently.

<img src="Prediction for User-Item Collaborative Filtering.png">

When making a prediction for **item-based CF** you don't need to correct for users average rating.

<img src="Prediction for Item-Item Collaborative Filtering.png">

In [16]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [17]:
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')

In [18]:
print(user_prediction.shape)
user_prediction

(943, 1682)


array([[ 1.53268928,  0.55940825,  0.45802095, ...,  0.26431948,
         0.26681188,  0.26678143],
       [ 1.29154594,  0.30818839,  0.15435664, ..., -0.06347986,
        -0.06015347, -0.06002952],
       [ 1.32150063,  0.27299383,  0.1307153 , ..., -0.09486531,
        -0.0915758 , -0.09147853],
       ...,
       [ 1.17286156,  0.2315538 ,  0.08923761, ..., -0.12132699,
        -0.11804445, -0.1180412 ],
       [ 1.32485211,  0.30688649,  0.18407549, ..., -0.03534088,
        -0.03252709, -0.03208789],
       [ 1.37623381,  0.38179911,  0.28246489, ...,  0.08935504,
         0.09196001,  0.09180163]])

In [19]:
print(item_prediction.shape)
item_prediction

(943, 1682)


array([[0.34126502, 0.35334049, 0.37247024, ..., 0.4164188 , 0.40895267,
        0.40822318],
       [0.08789664, 0.1034141 , 0.09884502, ..., 0.10588935, 0.10586744,
        0.10620214],
       [0.07209684, 0.0761115 , 0.07425959, ..., 0.07555027, 0.07538772,
        0.07612148],
       ...,
       [0.03148478, 0.03990949, 0.03939046, ..., 0.04521118, 0.04489761,
        0.04504912],
       [0.10913505, 0.11665775, 0.1216822 , ..., 0.12671029, 0.1245429 ,
        0.12702011],
       [0.19433364, 0.19140905, 0.21102847, ..., 0.24211779, 0.2372025 ,
        0.23540024]])

<h3><center>Evaluate the *accuracy* of predicted ratings using *Root Mean Squared Error (RMSE)*</center></h3>

You can use the mean_square_error (MSE) function from sklearn, where the RMSE is just the square root of MSE. To read more about different evaluation metrics you can take a look at this article: http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf

<img src="Root Mean Squared Error (RMSE).png">

Since you only want to consider predicted ratings that are in the test dataset, you filter out all other elements in the prediction matrix with prediction[ground_truth.nonzero()].

In [20]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [21]:
print('User-based CF RMSE:', rmse(user_prediction, test_data_matrix))
print('Item-based CF RMSE:', rmse(item_prediction, test_data_matrix))

User-based CF RMSE: 3.1335829812120517
Item-based CF RMSE: 3.45922045958584


**Memory-based** algorithms are easy to implement and produce reasonable prediction quality. The drawback of **memory-based CF** is that it **doesn't scale to real-world scenarios** and doesn't address the well-known **cold-start problem**, that is when new user or new item enters the system. 

Model-based CF methods are **scalable** and can deal with higher sparsity level than memory-based models, but also suffer when new users or items that don't have any ratings enter the system. The author would like to thank Ethan Rosenthal for his post about Memory-Based Collaborative Filtering.
http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/

<h3><center>Model-based Collaborative Filtering</center></h3>

Model-based Collaborative Filtering is based on **matrix factorization (MF)** which has received greater exposure, mainly as an **unsupervised learning** method for **latent variable decomposition** and **dimensionality reduction**. Matrix factorization is widely used for recommender systems where it can deal better with **scalability** and **sparsity** than Memory-based CF. 

The goal of MF is to learn the **latent preferences of users** and the **latent attributes of items** from **known ratings** (learn features that describe the characteristics of ratings) to then **predict the unknown ratings** through the dot product of the latent features of users and items. 

When you have a very sparse matrix, with a lot of dimensions, by doing matrix factorization you can restructure the user-item matrix into **low-rank structure**, and you can represent the matrix by the multiplication of two low-rank matrices, where the **rows contain the latent vector**. You **fit** this matrix to approximate your original matrix, as closely as possible, by multiplying the low-rank matrices together, which fills in the **entries missing** in the original matrix.

Let's calculate the sparsity level of MovieLens dataset:

In [33]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens100K is 93.7%


To give an example of the **learned latent preferences** of the **users** and **items**: let's say for the MovieLens dataset you have the following information: (user id, age, location, gender, movie id, director, actor, language, year, rating). 

By applying matrix factorization the model learns that important **user features** are age group (under 10, 10-18, 18-30, 30-90), location and gender, and for **movie features** it learns that *decade*, director and actor are most important. Now if you look into the information you have stored, there is no such feature as the *decade*, but the model can learn on its own. 

The important aspect is that the CF model only uses data (user_id, movie_id, rating) to learn the latent features. If there is little data available model-based CF model will predict poorly, since it will be more difficult to learn the latent features.



#### Hybrid Recommender Systems

Models that use both **ratings** and **content features** are called Hybrid Recommender Systems where both **Collaborative Filtering** and **Content-based Models** are combined. Hybrid recommender systems usually show higher accuracy than Collaborative Filtering or Content-based Models on their own: they are capable to address the cold-start problem better since if you don't have any ratings for a user or an item you could use the **metadata from the user or item** to make a prediction. 

#### Singular value decomposition (SVD)

A well-known matrix factorization method is Singular value decomposition (SVD). Collaborative Filtering can be formulated by **approximating a matrix X** by using singular value decomposition. The winning team at the Netflix Prize competition used SVD matrix factorization models to produce product recommendations. 

For more information I recommend to read articles: **Netflix Recommendations: Beyond the 5 stars** http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html and **Netflix Prize and SVD** http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf 

The general equation can be expressed as follows: X=U×S×VT

Given an m×n matrix X:
- U is an m×r orthogonal matrix
- S is an r×r diagonal matrix with non-negative real numbers on the diagonal
- VT is an r×n orthogonal matrix

Elements on the diagnoal in S are known as singular values of X

Matrix X can be factorized to U, S and V. 

The **U matrix** represents the feature vectors corresponding to the **users** in the hidden feature space.

The **V matrix** represents the feature vectors corresponding to the **items** in the hidden feature space.


<img src="Matrix X can be factorized to U S and V.png">

You can make a **prediction** by taking dot product of U, S and VT.

<img src="Prediction by dot product of U S and VT.png">

In [39]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE:', rmse(X_pred, test_data_matrix))

User-based CF MSE: 2.729073405113603


In [40]:
X_pred

array([[ 5.77826690e+00,  2.16217873e+00,  1.06209481e+00, ...,
         0.00000000e+00,  2.82980056e-02,  1.89304684e-02],
       [ 2.06904044e+00, -2.20221882e-01,  1.30978687e-01, ...,
         0.00000000e+00,  3.48985583e-03, -2.49903305e-02],
       [ 2.82244884e-01,  1.97196481e-02,  4.37889601e-02, ...,
         0.00000000e+00, -7.07344079e-03, -1.30560938e-03],
       ...,
       [ 2.41599002e+00, -3.86029712e-02,  2.18016182e-01, ...,
         0.00000000e+00,  1.87090022e-03, -8.33102486e-03],
       [ 3.27224234e-01,  1.43287055e-01, -1.81680928e-01, ...,
         0.00000000e+00,  1.59869165e-02, -2.91108906e-02],
       [ 7.67972310e-01,  1.50594299e+00,  6.80133871e-01, ...,
         0.00000000e+00,  2.70886235e-03,  2.39093120e-02]])

In [41]:
test_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [42]:
X_pred - test_data_matrix

array([[ 5.77826690e+00,  2.16217873e+00,  1.06209481e+00, ...,
         0.00000000e+00,  2.82980056e-02,  1.89304684e-02],
       [ 2.06904044e+00, -2.20221882e-01,  1.30978687e-01, ...,
         0.00000000e+00,  3.48985583e-03, -2.49903305e-02],
       [ 2.82244884e-01,  1.97196481e-02,  4.37889601e-02, ...,
         0.00000000e+00, -7.07344079e-03, -1.30560938e-03],
       ...,
       [ 2.41599002e+00, -3.86029712e-02,  2.18016182e-01, ...,
         0.00000000e+00,  1.87090022e-03, -8.33102486e-03],
       [ 3.27224234e-01,  1.43287055e-01, -1.81680928e-01, ...,
         0.00000000e+00,  1.59869165e-02, -2.91108906e-02],
       [ 7.67972310e-01, -3.49405701e+00,  6.80133871e-01, ...,
         0.00000000e+00,  2.70886235e-03,  2.39093120e-02]])

Carelessly addressing only the relatively few known entries is highly prone to overfitting. SVD can be very slow and computationally expensive. More recent work minimizes the squared error by applying alternating least square or stochastic gradient descent and uses regularization terms to prevent overfitting.

To wrap it up:
- In this post we have covered how to implement simple Collaborative Filtering methods, both memory-based CF and model-based CF.
- Memory-based models are based on similarity between items or users, where we use cosine-similarity.
- Model-based CF is based on matrix factorization where we use SVD to factorize the matrix.
- Building recommender systems that perform well in cold-start scenarios (where litle data is availabe on new users and items) remains a challenge. The standard collaborative filtering method performs poorly is such settings.